In [1]:
import os
import json
import py_vncorenlp
from tqdm.auto import tqdm

In [2]:
NOTEBOOK_DIR = os.getcwd()
CAPSTONE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATASET_DIR = os.path.join(CAPSTONE_DIR, "dataset")
MODEL_DIR = "D:/VnCoreNLP" # This is belong to your file location

In [3]:
model = py_vncorenlp.VnCoreNLP(save_dir= MODEL_DIR)

In [5]:
train_data_path = os.path.join(DATASET_DIR, "train_question_answer.json")
corpus_data_path = os.path.join(DATASET_DIR, "processed_legal_corpus.json")
output_path = os.path.join(DATASET_DIR, "processed_train_data.json")

with open(train_data_path, "r", encoding="utf-8") as json_file:
    train_data = json.load(json_file)
train_data_items = train_data["items"]

with open(corpus_data_path, "r", encoding="utf-8") as f:
    corpus_data = json.load(f)

total_items = len(train_data_items)

with open(output_path, "w", encoding="utf-8") as out_file:
    out_file.write("[\n")

    for i, item in enumerate(tqdm(train_data_items, total=total_items, desc="Processing training data")):
        relevant_articles = item["relevant_articles"]
        for article in relevant_articles:
            pair = []
            question = item["question"]
            question_tokens = model.word_segment(question)
            segmented_question = " ".join(question_tokens)
            pair.append(segmented_question)

            law_id = article["law_id"]
            article_id = article["article_id"]
            for law in corpus_data:
                if law["law_id"] == law_id:
                    for doc in law["articles"]:
                        if doc["article_id"] == article_id:
                            pair.append(doc["segment_only"])
                            break
        json.dump(pair, out_file, ensure_ascii=False, indent=4)
        if i < total_items - 1:
            out_file.write(",\n")
    out_file.write("\n]")




Processing training data:   0%|          | 0/3196 [00:00<?, ?it/s]